In [30]:
import sys
sys.executable

'/opt/homebrew/Caskroom/miniconda/base/envs/xdec/bin/python'

In [31]:
# 디코더의 인풋과 아웃풋 dim check
# 다음 토큰 1개? 마지막에 다음 토큰 1개를 포함하는 여러개?

In [32]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [33]:
batch_size = 16
seq_len = 32

dim_embed = 64
n_head = 4
n_layer = 4
dropout_ratio = 0.0

max_iters = 5000
learning_rate = 1e-3
eval_interval = 1000
eval_iters = 200

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [34]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

chars = sorted(list(set(text)))
vocab_size = len(chars)
print(vocab_size)

stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}

# input s is a string
encode = lambda s: [stoi[c] for c in s]
# input l is a list of integers
decode = lambda l: ''.join([itos[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

# dataloader
def get_batch(split):
    data = train_data if split == 'train' else val_data

    # randint of range [0, len(data)-seq_len]
    # shape of (batch_size,)
    ix = torch.randint(len(data)-seq_len, (batch_size,))
    x = torch.stack([data[i:i+seq_len] for i in ix])
    y = torch.stack([data[i+1:i+seq_len+1] for i in ix])
    # y는 오른쪽으로 한 칸 shifted된 텍스트 데이터임.
    x, y = x.to(device), y.to(device)
    return x, y

--2024-05-11 19:31:33--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  4.57MB/s    in 0.2s    

2024-05-11 19:31:34 (4.57 MB/s) - ‘input.txt’ saved [1115394/1115394]

65


In [35]:
class Head(nn.Module):

    """
    quick dim check

    B: batch_size 
    T: seq_len = token length = context length
    C: dim_embed = channels = n_head * head_size
    nh: n_head
    hs: head_size = dim_embed // n_head

    x: (B, T, C)
    
    W_k or W_q or W_v: (C, hs)
    
    k or q or v: (B, T, hs)
    
    att: (B, T, T)
    
    out: (B, T, hs)

    MHA 때문에 W_k or W_q or W_v의 크기가 (C, C)가 아니라 (C, hs)가 되어야 함.
    이래야 MHA를 거치고 나온 텐서의 크기가 (B, T, C)가 되니까.
    (nh개 만큼의 (B, T, hs)가 concat되어 (B, T, C)가 됨. (C = nh * hs))
    (Transformer를 구성하는 블럭들은 항상 (B, T, C) in, (B, T, C) out 구조.)
    """

    def __init__(self, head_size):
        super().__init__()
        self.W_k = nn.Linear(dim_embed, head_size, bias=False)
        self.W_q = nn.Linear(dim_embed, head_size, bias=False)
        self.W_v = nn.Linear(dim_embed, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(seq_len, seq_len)))

        self.dropout = nn.Dropout(dropout_ratio)

    def forward(self, x):
        B, T, C = x.shape # (B, T, C)
        k = self.W_k(x) # (B, T, hs)
        q = self.W_q(x) # (B, T, hs)

        att = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, hs) @ (B, hs, T) --> (B, T, T)
        att = att.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        att = F.softmax(att, dim=-1) # (B, T, T)
        att = self.dropout(att)

        v = self.W_v(x) # (B, T, hs)
        out = att @ v # (B, T, T) @ (B, T, hs) --> (B, T, hs)

        return out

class MultiHeadAttention(nn.Module):

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(dim_embed, dim_embed)
        self.dropout = nn.Dropout(dropout_ratio)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):

    def __init__(self, dim_embed):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(dim_embed, 4*dim_embed),
            nn.ReLU(),
            nn.Linear(4*dim_embed, dim_embed),
            nn.Dropout(dropout_ratio),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):

    def __init__(self, dim_embed, n_head):
        super().__init__()
        head_size = dim_embed // n_head
        self.mha = MultiHeadAttention(n_head, head_size)
        self.ff = FeedFoward(dim_embed)
        self.ln1 = nn.LayerNorm(dim_embed)
        self.ln2 = nn.LayerNorm(dim_embed)

    def forward(self, x):
        x = x + self.mha(self.ln1(x)) # communication
        x = x + self.ff(self.ln2(x)) # computation
        return x

class GPTLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        self.tokedim_embedding_table = nn.Embedding(vocab_size, dim_embed)
        self.positiodim_embedding_table = nn.Embedding(seq_len, dim_embed)
        self.blocks = nn.Sequential(*[Block(dim_embed, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(dim_embed)
        self.lm_head = nn.Linear(dim_embed, vocab_size)

        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None): # idx: (B, T)
        B, T = idx.shape

        tok_emb = self.tokedim_embedding_table(idx) # (B, T, C)
        pos_emb = self.positiodim_embedding_table(torch.arange(T, device=device)) # (T, C)

        x = tok_emb + pos_emb # (B, T, C)
        x = self.blocks(x) # (B, T, C)
        x = self.ln_f(x) # (B, T, C) 

        # self.lm_head = nn.Linear(dim_embed, vocab_size)
        logits = self.lm_head(x) # (B, T, vocab_size)
        
        """
        계산된 logits의 dim은 (B, T, vocab_size)다.
        (B, 1, vocab_size)가 아님.
        target으로 넣어주는 값이 오른쪽으로 한 칸 shifted된 문장이라서 그래야함.

        그럼 왜 인풋 토큰 시퀀스의 다음에 올 토큰만 예측하지 않고
        다음 한 토큰으로 shifted된 토큰 시퀀스 전체를 예측하나?
        그래야 y-hat과 비교할 y가 많아져서 학습이 효율적이라서?
        """

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens): # idx: (B, T)
        for _ in range(max_new_tokens):
            
            # context length에 맞게 자른다
            # because T was lengthened by the previous generation
            idx_cond = idx[:, -seq_len:]
            # idx_cond.shape: torch.Size([1, 1])
            # idx_cond.shape: torch.Size([1, 2])
            # ...
            # idx_cond.shape: torch.Size([1, 32])

            logits, loss = self(idx_cond)
            # logits.shape: torch.Size([1, 1, 65])
            # logits.shape: torch.Size([1, 2, 65])
            # ...
            # logits.shape: torch.Size([1, 32, 65])

            """
            인풋으로 idx가 하나만 들어가면 logit 아웃풋도 하나만 나옴.
            구조상 항상 seq_len만큼의 토큰이 들어가고 나와야 하는 것 아님?
            이게 masked와 연관이 있나?
            """

            # focus only on the last time step
            logits = logits[:, -1, :] # (B, T, vocab_size) --> (B, vocab_size)
            # logits.shape: torch.Size([1, 65])

            probs = F.softmax(logits, dim=-1)
            # probs.shape: torch.Size([1, 65])

            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # idx_next.shape: torch.Size([1, 1])

            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [36]:
model = GPTLanguageModel()
m = model.to(device)
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out
     

0.209729 M parameters


In [37]:
for iter in range(max_iters+1):

    if iter % eval_interval == 0 or iter == max_iters:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    xb, yb = get_batch('train')

    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=1000)[0].tolist()))

step 0: train loss 4.2140, val loss 4.2130
step 1000: train loss 2.0553, val loss 2.0994
step 2000: train loss 1.8505, val loss 1.9517
step 3000: train loss 1.7552, val loss 1.8882
step 4000: train loss 1.6931, val loss 1.8734
step 5000: train loss 1.6587, val loss 1.8230
torch.Size([1, 1])
torch.Size([1, 2])
torch.Size([1, 3])
torch.Size([1, 4])
torch.Size([1, 5])
torch.Size([1, 6])
torch.Size([1, 7])
torch.Size([1, 8])
torch.Size([1, 9])
torch.Size([1, 10])
torch.Size([1, 11])
torch.Size([1, 12])
torch.Size([1, 13])
torch.Size([1, 14])
torch.Size([1, 15])
torch.Size([1, 16])
torch.Size([1, 17])
torch.Size([1, 18])
torch.Size([1, 19])
torch.Size([1, 20])
torch.Size([1, 21])
torch.Size([1, 22])
torch.Size([1, 23])
torch.Size([1, 24])
torch.Size([1, 25])
torch.Size([1, 26])
torch.Size([1, 27])
torch.Size([1, 28])
torch.Size([1, 29])
torch.Size([1, 30])
torch.Size([1, 31])
torch.Size([1, 32])
torch.Size([1, 32])
torch.Size([1, 32])
torch.Size([1, 32])
torch.Size([1, 32])
torch.Size([1, 3

$L = -\sum_{i=1}^{\text{num\_classes}} t_i \log(p_i)$

$L = -\log\left(\frac{1}{\text{num\_classes}}\right)$

$L = \log(\text{num\_classes})$

In [38]:
# step 0: train loss 4.1716, val loss 4.1712

torch.log(torch.tensor(65.0))

tensor(4.1744)

In [41]:
"""
MHA dim check utilizing tensor concat

B: batch_size 
T: seq_len = token length = context length
C: dim_embed = channels = n_head * head_size
nh: n_head
hs: head_size = dim_embed // n_head
"""

B, T, C = 2, 3, 4
x = torch.randn(B, T, C)
print(f"x.shape: {x.shape}") # (B, T, C)

# W_q, W_k, W_v concatted into one
W_qkv = torch.randn(C, 3*C) 
print(f"W_qkv.shape: {W_qkv.shape}") # (C, 3C)

qkv = torch.matmul(x, W_qkv)
print(f"qkv.shape: {qkv.shape}") # (B, T, 3C)

# q, k, v를 분리해내고
q, k, v = qkv.split(C, dim=2)
print(f"q.shape: {q.shape}") # (B, T, C)
print(f"k.shape: {k.shape}") # (B, T, C)
print(f"v.shape: {v.shape}") # (B, T, C)

nh = 2
# head별로 다시 나눔
# 3d tensor --> 4d tensor
q = q.view(B, T, nh, C//nh) 
k = k.view(B, T, nh, C//nh) 
v = v.view(B, T, nh, C//nh) 
print(f"q.shape: {q.shape}") # (B, T, nh, hs)
print(f"k.shape: {k.shape}") # (B, T, nh, hs)
print(f"v.shape: {v.shape}") # (B, T, nh, hs)

q = q.transpose(1, 2) 
k = k.transpose(1, 2)
v = v.transpose(1, 2)
print(f"q.shape: {q.shape}") # (B, nh, T, hs)
print(f"k.shape: {k.shape}") # (B, nh, T, hs)
print(f"v.shape: {v.shape}") # (B, nh, T, hs)

# att = q @ k.T
att = q @ k.transpose(-2, -1) # (B, nh, T, hs) @ (B, nh, hs, T) --> (B, nh, T, T)
print(f"att.shape: {att.shape}") # (B, nh, T, T)

att = att * (1.0 / torch.sqrt(torch.tensor(k.size(-1), dtype=torch.float32)))
att = F.softmax(att, dim=-1)

# y = att @ v
y = att @ v # (B, nh, T, T) @ (B, nh, T, hs) --> (B, nh, T, hs)
print(f"y.shape: {y.shape}") # (B, nh, T, hs)

# 나눴던 head들을 다시 붙임
y = y.transpose(1, 2).contiguous().view(B, T, C)
print(f"y.shape: {y.shape}") # (B, T, C)

x.shape: torch.Size([2, 3, 4])
W_qkv.shape: torch.Size([4, 12])
qkv.shape: torch.Size([2, 3, 12])
q.shape: torch.Size([2, 3, 4])
k.shape: torch.Size([2, 3, 4])
v.shape: torch.Size([2, 3, 4])
q.shape: torch.Size([2, 3, 2, 2])
k.shape: torch.Size([2, 3, 2, 2])
v.shape: torch.Size([2, 3, 2, 2])
q.shape: torch.Size([2, 2, 3, 2])
k.shape: torch.Size([2, 2, 3, 2])
v.shape: torch.Size([2, 2, 3, 2])
att.shape: torch.Size([2, 2, 3, 3])
y.shape: torch.Size([2, 2, 3, 2])
y.shape: torch.Size([2, 3, 4])
